## Workshop Week 6

## Logistic Regression
Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [143]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings('ignore')

In [144]:
bcancer = pd.read_csv("files/breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [145]:
# Examine the data: check number of rows and number of columns
bcancer.shape

(699, 11)

In [146]:
# Look at the statistical summary of the dataframe
bcancer.describe()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,683.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.643857,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [147]:
# Check how many classes we do have from the "class" column
bcancer['class'].unique()
set(bcancer['class'])

{2, 4}

In [148]:
# Check number of samples for each class and comment whether dataset is balanced?
print("No. of benign samples: ", bcancer[bcancer['class'] == 2].shape[0])
print("No. of malignant samples: ", bcancer[bcancer['class'] == 4].shape[0])

No. of benign samples:  458
No. of malignant samples:  241


In [149]:
# Deal with the NaN values in the data
bcancer.isna().sum()
bcancer = bcancer.dropna()

In [150]:
# check shape again
bcancer.shape

(683, 11)

In [151]:
# Split your data into training(80%) and testing data (20%) and use random_state=142
train, test = train_test_split(bcancer, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(546, 11)
(137, 11)


In [152]:
#Training Logistic Regression model
X_train = train.drop(columns=['class', 'sample_code_number'])
y_train = train['class']
X_test = test.drop(columns=['class', 'sample_code_number'])
y_test = test['class']
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [153]:
# Do predictions on test set
y_test_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
print('y = X * ', reg.coef_, '+', reg.intercept_)
print('MSE: ', mean_squared_error(y_test, y_test_pred))
print('R2: ', r2_score(y_test, y_test_pred))

y = X *  [ 0.06317386  0.04985229  0.01422923  0.0206944   0.02328527  0.08855803
  0.04763378  0.03429389 -0.00671935] + 1.4996256653609557
MSE:  0.1684017215978665
R2:  0.8212255705503192


### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [154]:
# Evaluate the performance of your trained model
# error message: Can't Handle mix of binary and continuous target, y_train_pred replaced by np.round(abs(y_train_pred))
print(accuracy_score(y_train, np.round(abs(y_train_pred))))
print(accuracy_score(y_test, np.round(abs(y_test_pred))))

0.8644688644688645
0.8394160583941606


In [203]:
# Checking confusion matrix
# print(y_test)
# print(y_test_pred)
print('Confusion matrix on test set: ')
print(confusion_matrix(y_test, np.round(abs(y_test_pred)), labels=[1,2,3,4]))

Confusion matrix on test set: 
[[ 0  0  0  0]
 [ 0 82  3  0]
 [ 0  0  0  0]
 [ 0  0 16 33]]


In [ ]:
### Analysis
- the Accuracy Score of both train and test dataset is higher than 80 %, which means the model can generally predict the result with more than 80% chance right.
- as can be seen from Confusion Matrix, 16+3 are false predictions, among which highest false prediction type account for 16, while 82 + 33 are true predictions, 
which means the possibility to correct prediction is high.

**This is the checkpoint mark for this week's workshop. You need to report `Accuracy Score` on test set and also show `confusion matrix`. You also need to provide analysis based on the results you got.**

### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [162]:
# Creating RFE object
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=3, step=1)
rfe.fit(X_train, y_train)

RFE(estimator=LogisticRegression(), n_features_to_select=3)

In [164]:
# summarize all features ('features' means 'columns')
for i in range(X_train.shape[1]):
    print('Column: %d, selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, selected True, Rank: 1.000
Column: 1, selected False, Rank: 7.000
Column: 2, selected True, Rank: 1.000
Column: 3, selected False, Rank: 2.000
Column: 4, selected False, Rank: 6.000
Column: 5, selected False, Rank: 3.000
Column: 6, selected True, Rank: 1.000
Column: 7, selected False, Rank: 5.000
Column: 8, selected False, Rank: 4.000


### Note:
- model with 5 features: column 1, 4, 7, 8 will be dropped.(n_features_to_select=5)
- model with 4 features: column 1, 4, 5, 7, 8 will be dropped.(n_features_to_select=4)
- model with 3 features: column 1, 3, 4, 5, 7, 8 will be dropped.(n_features_to_select=3)

In [166]:
X_train.head(1)

,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses
566,3,1,2,1,2,1.0,3,1,1


In [183]:
#Training Logistic Regression model with 5 features
X_train = train.drop(columns=['class', 'sample_code_number', 'uniformity_cell_size', 'single_epithelial_cell_size', 'normal_nucleoli', 'mitoses'])
y_train = train['class']
X_test = test.drop(columns=['class', 'sample_code_number', 'uniformity_cell_size', 'single_epithelial_cell_size', 'normal_nucleoli', 'mitoses'])
y_test = test['class']
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

280    2
232    2
369    2
563    2
491    4
      ..
645    2
353    4
307    2
126    4
67     4
Name: class, Length: 137, dtype: int64


LinearRegression()

In [168]:
# Do predictions on test set
y_test_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
print('y = X * ', reg.coef_, '+', reg.intercept_)
print('MSE: ', mean_squared_error(y_test, y_test_pred))
print('R2: ', r2_score(y_test, y_test_pred))

y = X *  [0.07282788 0.07191594 0.03631298 0.08995228 0.06229424] + 1.4941816418841718
MSE:  0.1778417662508893
R2:  0.8112040661333179


In [169]:
#Training Logistic Regression model with 4 features
X_train = train.drop(columns=['class', 'sample_code_number', 'uniformity_cell_size', 'single_epithelial_cell_size', 'normal_nucleoli', 'mitoses', 'bland_chromatin'])
y_train = train['class']
X_test = test.drop(columns=['class', 'sample_code_number', 'uniformity_cell_size', 'single_epithelial_cell_size', 'normal_nucleoli', 'mitoses', 'bland_chromatin'])
y_test = test['class']
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [170]:
# Do predictions on test set
y_test_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
print('y = X * ', reg.coef_, '+', reg.intercept_)
print('MSE: ', mean_squared_error(y_test, y_test_pred))
print('R2: ', r2_score(y_test, y_test_pred))

y = X *  [0.07721725 0.09033166 0.04875549 0.10072895] + 1.5584403138314158
MSE:  0.183751494789519
R2:  0.804930327731647


In [171]:
#Training Logistic Regression model with 3 features
X_train = train.drop(columns=['class', 'sample_code_number', 'uniformity_cell_size', 'single_epithelial_cell_size', 'normal_nucleoli', 'mitoses', 'bland_chromatin', 'marginal_adhesion'])
y_train = train['class']
X_test = test.drop(columns=['class', 'sample_code_number', 'uniformity_cell_size', 'single_epithelial_cell_size', 'normal_nucleoli', 'mitoses', 'bland_chromatin', 'marginal_adhesion'])
y_test = test['class']
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [172]:
# Do predictions on test set
y_test_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
print('y = X * ', reg.coef_, '+', reg.intercept_)
print('MSE: ', mean_squared_error(y_test, y_test_pred))
print('R2: ', r2_score(y_test, y_test_pred))

y = X *  [0.07491064 0.10820585 0.11749688] + 1.5889851384674158
MSE:  0.18025622123642215
R2:  0.8086408927383254


## Conclusion

Write a brief conclusion to your experiment.  You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 

- The model with all features(clump_thickness, uniformity_cell_size, uniformity_cell_shape, marginal_adhesion, single_epithelial_cell_size, bare_nuclei, bland_chromatin, normal_nucleoli,mitoses) get the best MSE:  0.1684017215978665
R2:  0.8212255705503192. So all features should be taken into account when making prediction of class of breast cancer. Any features missing will lower the accuracy of prediction.
- As can be seen from confusion metrix, there are 82 cases are correctly predicted as benign (good), while 3 cases(acturally good) are predicted unclear because the prediction result is between benign (good) and malignant (bad);
- There are 33 cases are correctly predicted as malignant (bad), while 33 cases (acturally bad) are predicted unclear because the prediction result is between benign (good) and malignant (bad).
- No malignant (bad) cases are predicted as benign (good), and vice versa.